In [77]:
import pandas as pd
from pandasgui import show

In [78]:
df_prices = pd.read_excel('Product Lookup.xlsx')
df_customers = pd.read_excel('Customer Information.xlsx')
df_area_codes = pd.read_excel('Area Code Lookup.xlsx')

In [79]:
df_customers = df_customers['IDs'].str.split().explode().reset_index(drop=True).to_frame()

In [80]:
extract_regex = r'\D*?(?P<phone>\d{6}).*?\,(?P<join>\d{2}\w)(?P<ordered>\d+)-(?P<product_id>.+)'
df_orders = df_customers['IDs'].str.extract(extract_regex)
df_orders['ordered'] = df_orders['ordered'].astype(int)
# df_orders['join'] = df_orders['join'].astype('string')
df_orders.rename(columns={'product_id':'Product ID'}, inplace=True)
df_orders

,phone,join,ordered,Product ID
0,856452,13S,24,SPL
1,234175,29M,77,SPL
2,621372,42K,26,SBP
3,559207,53K,50,HS
4,391563,00C,69,SPL
...,...,...,...,...
994,723726,09H,15,SBP
995,098928,28F,45,SPL
996,833098,41A,65,HS
997,961094,52A,25,SBP


In [81]:
df_area_codes['join'] = df_area_codes['Code'].astype('string').str[-2:] + df_area_codes['Area'].str[:1].str.upper()

In [124]:
df_area_codes['join count'] = df_area_codes.groupby('join')['join'].transform('count')
unwanted_cities = ['Clevedon', 'Fakenham', 'Stornoway']
df_area_codes = df_area_codes[(df_area_codes['join count'] == 1) & (~df_area_codes['Area'].isin(unwanted_cities))]
df_area_codes

,Code,Area,join,join count
0,114,Sheffield,14S,1
1,115,Nottingham,15N,1
2,116,Leicester,16L,1
3,117,Bristol,17B,1
4,118,Reading,18R,1
...,...,...,...,...
740,2885,Ballygawley,85B,1
743,2889,Fivemiletown,89F,1
745,2891,Bangor (Co. Down),91B,1
747,2893,Ballyclare,93B,1


In [126]:
df_output = pd.merge(df_orders, df_area_codes, on='join', how='left').groupby(['Area', 'Product ID'], as_index=False)['ordered'].sum().merge(df_prices, on='Product ID', how='left')

In [127]:
df_output['Price'] = df_output['Price'].str.strip('£').astype('float')
df_output['ordered'] = df_output['ordered'].astype('int')
df_output['Revenue'] = df_output['ordered'] * df_output['Price']
df_output['Rank'] = df_output.groupby('Area')['Revenue'].rank()

df_output['area rev'] = df_output.groupby('Area')['Revenue'].transform('sum')
df_output['% of tot rev for the area'] = df_output['Revenue'] / df_output['area rev']
show(df_output)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
